In [56]:
import os
import sys
import cv2
import time
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras import backend as K
from common import distance, functions
from models import Facenet
from detectors import FaceDetector
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [ ]:
ROOT = "../data/FaceData/"
path_haar = "../data/haarcascade_frontalface_default.xml"


def read_image(index):
    path = os.path.join(ROOT, index[0], index[1])
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (160, 160))
    return image

In [ ]:
def split_dataset(directory, split=0.9):
    folders = os.listdir(directory)
    num_train = int(len(folders) * split)

    random.shuffle(folders)

    train_list, test_list = {}, {}

    # Creating Train-list
    for folder in folders[:num_train]:
        num_files = len(os.listdir(os.path.join(directory, folder)))
        train_list[folder] = num_files

    # Creating Test-list
    for folder in folders[num_train:]:
        num_files = len(os.listdir(os.path.join(directory, folder)))
        test_list[folder] = num_files

    return train_list, test_list


train_list, test_list = split_dataset(ROOT, split=0.9)
print("Length of training list:", len(train_list))
print("Length of testing list :", len(test_list))

# train_list, test list contains the folder names along with the number of files in the folder.
print("\nTest List:", test_list)

In [ ]:
def create_triplets(directory, folder_list, max_files=10):
    triplets = []
    folders = list(folder_list.keys())

    for folder in folders:
        path = os.path.join(directory, folder)
        files = list(os.listdir(path))[:max_files]
        num_files = len(files)

        for i in range(num_files - 1):
            for j in range(i + 1, num_files):
                anchor = (folder, f"{files[i]}")
                positive = (folder, f"{files[j]}")

                neg_folder = folder
                while neg_folder == folder:
                    neg_folder = random.choice(folders)
                neg_file = random.choice(
                    os.listdir(os.path.join(directory, neg_folder))
                )
                negative = (neg_folder, f"{neg_file}")

                triplets.append((anchor, positive, negative))

    random.shuffle(triplets)
    return triplets

In [ ]:
train_triplet = create_triplets(ROOT, train_list)
test_triplet = create_triplets(ROOT, test_list)

print("Number of training triplets:", len(train_triplet))
print("Number of testing triplets :", len(test_triplet))

print("\nExamples of triplets:")
for i in range(5):
    print(train_triplet[i])

In [ ]:
def get_batch(triplet_list, batch_size=256, preprocess=True):
    while True:
        i = 0
        anchor = []
        positive = []
        negative = []

        while i < batch_size:
            a, p, n = random.choice(triplet_list)

            anc_img = np.array(read_image(a))
            pos_img = np.array(read_image(p))
            neg_img = np.array(read_image(n))

            if preprocess:
                anc_img = preprocess_input(anc_img)
                pos_img = preprocess_input(pos_img)
                neg_img = preprocess_input(neg_img)

            anchor.append(anc_img)
            positive.append(pos_img)
            negative.append(neg_img)

            i += 1

        yield (
            [np.array(anchor), np.array(positive), np.array(negative)],
            np.zeros((batch_size, 1)).astype("float32"),
        )

In [ ]:
num_plots = 6

f, axes = plt.subplots(num_plots, 3, figsize=(15, 20))

for x in get_batch(train_triplet, batch_size=num_plots, preprocess=False):
    a, p, n = x[0][0], x[0][1], x[0][2]
    for i in range(num_plots):
        axes[i, 0].imshow(a[i])
        axes[i, 1].imshow(p[i])
        axes[i, 2].imshow(n[i])
        i += 1
    break

In [ ]:
def triplet_loss_t(y_true, y_pred):
    anchor = y_pred[:, 0:128]
    pos = y_pred[:, 128:256]
    neg = y_pred[:, 256:384]

    positive_distance = K.sum(K.abs(anchor - pos), axis=1)
    negative_distance = K.sum(K.abs(anchor - neg), axis=1)

    # Concatenate positive_distance and negative_distance along a new axis
    distances = K.stack([positive_distance, negative_distance], axis=1)

    # Apply softmax to the concatenated tensor along axis 1
    probs = K.softmax(distances, axis=1)

    # Calculate the loss using the probabilities
    loss = K.mean(K.abs(probs[:, 0]) + K.abs(1.0 - probs[:, 1]))
    return loss

In [57]:
pretrained_model = Facenet.loadModel()
pretrained_model.trainable = True

fine_tune_at = 355
for layer in pretrained_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
last_layer = pretrained_model.get_layer("Dropout")
last_output = last_layer.output
# Bottleneck
x = tf.keras.layers.Dense(256, use_bias=False, name="Bottleneck")(last_output)
x = tf.keras.layers.BatchNormalization(
    momentum=0.995, epsilon=0.001, scale=False, name="Bottleneck_BatchNorm"
)(x)

model = tf.keras.models.Model(pretrained_model.input, x)

In [ ]:
model.summary()

In [ ]:
triplet_model_a = tf.keras.layers.Input((160, 160, 3))
triplet_model_n = tf.keras.layers.Input((160, 160, 3))
triplet_model_p = tf.keras.layers.Input((160, 160, 3))
triplet_model_out = tf.keras.layers.Concatenate()(
    [model(triplet_model_a), model(triplet_model_p), model(triplet_model_n)]
)
triplet_model = tf.keras.models.Model(
    [triplet_model_a, triplet_model_p, triplet_model_n], triplet_model_out
)

In [ ]:
triplet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=triplet_loss_t)

In [ ]:
triplet_model.summary()

In [ ]:
history = triplet_model.fit(get_batch(train_triplet), validation_data=get_batch(test_triplet), steps_per_epoch=100, epochs=150)

In [ ]:
triplet_model.save("triplet_model.h5")